Este código está utilizando la computación cuántica para resolver un problema de optimización. Aquí está lo que hace cada parte:

1. Importa las bibliotecas necesarias para interactuar con el hardware cuántico de D-Wave y para construir el modelo de optimización.

2. Configura el tipo de solver y el token de autenticación para interactuar con el hardware de D-Wave.

3. Inicializa el solver con la configuración especificada.

4. Define los parámetros del modelo. En este caso, `mu` es un parámetro del modelo y `N` es el tamaño del tablero de Go.

5. Crea variables de spin para cada posición en el tablero de Go. En física, el "spin" es una propiedad cuántica que puede ser utilizada para representar estados binarios.

6. Define un valor ficticio para `h`, que representa el número de libertades de cada posición en el tablero.

7. Construye el Hamiltoniano Ising, que es una representación matemática del problema de optimización. En este caso, el Hamiltoniano incluye interacciones entre vecinos y un término que representa las libertades.

8. Crea un modelo binario cuadrático a partir del Hamiltoniano.

9. Crea un sampler, que es un objeto que puede encontrar la solución óptima al problema de optimización.

10. Muestra el resultado de la optimización utilizando el inspector de D-Wave.

En resumen, este código está utilizando la computación cuántica para encontrar la configuración óptima de un tablero de Go, dadas ciertas restricciones y parámetros.

In [56]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 19

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Suponiendo que h_i(t) se puede calcular o se da como input. Aquí usamos un valor ficticio
# Por ejemplo, todos tienen 4 libertades inicialmente (completamente abierto)
h = {(i, j): 4 for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
h_values = {}
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1
        
        # Añadir el término de libertades
        h_values[spins[(i, j)]] = -mu * h[(i, j)]

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)



Serving Inspector on http://127.0.0.1:18000/?problemId=eb1a2565-cd6a-42e8-9d84-bd06659426f9

'http://127.0.0.1:18000/?problemId=eb1a2565-cd6a-42e8-9d84-bd06659426f9'

Este código es muy similar al anterior, pero con una diferencia clave: este código establece una configuración inicial para el tablero de Go.

En el código anterior, todas las posiciones del tablero se inicializan con el mismo valor. En este código, sin embargo, se establece una configuración inicial específica para el tablero. En particular, se coloca una piedra blanca en la posición (1, 1) y una piedra negra en la posición (4, 4).

Además, este código configura los valores de `h_values` de acuerdo con el estado inicial del tablero. En el código anterior, `h_values` se configura en función de un valor ficticio que representa el número de libertades de cada posición en el tablero.

En resumen, este código tiene en cuenta una configuración inicial específica del tablero de Go, mientras que el código anterior no lo hace. Ambos códigos utilizan la computación cuántica para encontrar la configuración óptima del tablero de Go, dadas ciertas restricciones y parámetros.

In [57]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board = {(i, j): 0 for i in range(N) for j in range(N)}
initial_board[(1, 1)] = 1  # Piedra blanca
initial_board[(4, 4)] = -1  # Piedra negra

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: initial_board[(i, j)] for i in range(N) for j in range(N)}

# Suponiendo que h_i(t) se puede calcular o se da como input. Aquí usamos un valor ficticio
# Por ejemplo, todos tienen 4 libertades inicialmente (completamente abierto)
h = {(i, j): 4 for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=d7516508-f0cd-4910-b48b-36eff1e52c1d

'http://127.0.0.1:18000/?problemId=d7516508-f0cd-4910-b48b-36eff1e52c1d'

Experimento ocupando en el ISING una sigmoide transformada
Comparación entre este código y el anterior
Ambos códigos están diseñados para resolver problemas de optimización cuántica utilizando el modelo Ising y el solver D-Wave. Sin embargo, hay varias diferencias clave entre ellos:

1. **Configuración inicial del tablero**: En el primer código, el tablero inicial se configura manualmente con un diccionario y se establecen dos posiciones con piedras blancas y negras. En el segundo código, se utiliza `numpy.zeros` para crear un tablero inicial y también se establecen dos posiciones con piedras blancas y negras, pero en diferentes posiciones que en el primer código.

2. **Cálculo de h_values**: En el primer código, `h_values` se configura simplemente en base al estado inicial del tablero. En el segundo código, se utiliza una función más compleja `compute_local_energy` para calcular `h_values`, que toma en cuenta la energía local de cada posición en el tablero. Esta función considera las libertades de cada posición (es decir, el número de posiciones adyacentes vacías) y las transforma utilizando una función sigmoide.

3. **Impresión de respuestas**: El segundo código incluye una función `print_all_responses` que imprime todas las respuestas del sampler, mientras que el primer código no tiene esta funcionalidad.

4. **Cambio de tamaño del tablero**: El segundo código incluye una función `change_board_size` que permite cambiar el tamaño del tablero. Esta función actualiza el tablero inicial, las variables de spin y `h_values` para adaptarse al nuevo tamaño del tablero. El primer código no tiene esta funcionalidad.

En resumen, aunque ambos códigos están diseñados para resolver el mismo tipo de problema, el segundo código es más sofisticado y flexible, permitiendo un cálculo más detallado de `h_values` y la posibilidad de cambiar el tamaño del tablero.

In [60]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board = np.zeros((N, N))
initial_board[(2, 2)] = 1  # Piedra blanca
initial_board[(3, 3)] = -1  # Piedra negra

# Funciones del primer script adaptadas para calcular la energía y las libertades
def sigmoid_transformed(x):
    return 1 / (1 + np.exp(-1 / np.sqrt(x + 0.0001)))

def get_neighbors_positions(i, j, BOARD_SIZE):
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def compute_liberties(board, i, j, BOARD_SIZE):
    empty_spots = [1 if board[k][l] == 0 else 0 for k, l in get_neighbors_positions(i, j, BOARD_SIZE) if k >= 0 and l >= 0 and k < BOARD_SIZE and l < BOARD_SIZE]
    return sum(empty_spots)

def compute_local_energy(board, i, j, mu, BOARD_SIZE):
    xi = board[i][j]
    neighbors = [board[x][y] for x, y in get_neighbors_positions(i, j, BOARD_SIZE) if x >= 0 and y >= 0 and x < BOARD_SIZE and y < BOARD_SIZE]
    liberties = compute_liberties(board, i, j, BOARD_SIZE)
    sigmoid_liberties_effect = sigmoid_transformed(liberties)
    return xi * sum(neighbors) - mu * sigmoid_liberties_effect

# Configurar h_values de acuerdo con el estado inicial del tablero y la energía local
h_values = {spins[(i, j)]: compute_local_energy(initial_board, i, j, mu, N) for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear el modelo
response = sampler.sample(model)

# Función para imprimir todas las respuestas
def print_all_responses(response):
    for sample, energy in response.data(['sample', 'energy']):
        print(f"Configuración: {sample}, Energía: {energy}")

print_all_responses(response)

# Función para cambiar el tamaño del tablero
def change_board_size(new_size):
    global N
    N = new_size
    global initial_board
    initial_board = np.zeros((N, N))
    global spins
    spins = {(i, j): i*N+j for i in range(N) for j in range(N)}
    global h_values
    h_values = {spins[(i, j)]: compute_local_energy(initial_board, i, j, mu, N) for i in range(N) for j in range(N)}

# Cambiar el tamaño del tablero a 9
change_board_size(9)

# Mostrar el inspector
ins.show(response)

Configuración: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1}, Energía: -96.15581354755936


Serving Inspector on http://127.0.0.1:18000/?problemId=cb0a2da4-0271-4d64-aabd-09ba6357bed4

'http://127.0.0.1:18000/?problemId=cb0a2da4-0271-4d64-aabd-09ba6357bed4'

OIsing con muchos parámetros sin correlacionar
Este código está diseñado para resolver un problema de optimización en el juego de Go utilizando un ordenador cuántico D-Wave. Aquí está lo que hace cada parte:

1. Importa las bibliotecas necesarias.

2. Configura el solver D-Wave con un token y un tipo de solver específicos.

3. Define los parámetros del modelo, incluyendo `mu` y el tamaño del tablero `N`.

4. Crea una representación de las posiciones en el tablero de Go como variables de spin.

5. Define el estado inicial del tablero, con una piedra blanca en la posición (2,2) y una piedra negra en la posición (3,3).

6. Define varias funciones para calcular la energía local y las libertades de una posición en el tablero.

7. Configura los valores de `h` para cada posición en el tablero, basándose en la energía local.

8. Construye el Hamiltoniano Ising, que representa la energía del sistema, incluyendo las interacciones entre las posiciones vecinas en el tablero.

9. Crea un modelo binario cuadrático (BQM) a partir de los valores de `h` y `J`, y un tipo de variable de spin.

10. Crea un sampler, que es un objeto que puede muestrear (es decir, encontrar soluciones a) el BQM.

11. Muestrea el BQM y guarda la respuesta.

12. Define una función para imprimir todas las respuestas (es decir, las soluciones al BQM y sus energías correspondientes).

13. Imprime todas las respuestas.

14. Define una función para cambiar el tamaño del tablero, que actualiza `N`, `initial_board`, `spins` y `h_values` en consecuencia.

15. Cambia el tamaño del tablero a 9.

16. Muestra el inspector D-Wave, que proporciona una interfaz gráfica para visualizar la respuesta.

In [11]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
params = {
    'J': np.random.uniform(0.1, 1.0),  # Interacción entre piedras adyacentes.
    'J_ij': np.random.uniform(0.1, 1.0),  # Interacción a larga distancia.
    'alpha': np.random.uniform(0.1, 1.0),  # Coeficiente de decaimiento para la interacción a larga distancia.
    'L': np.random.uniform(0.1, 1.0),  # Importancia de las situaciones de Ko y Seki.
    'M': np.random.uniform(0.1, 1.0),  # Importancia de las condiciones de borde.
    'R': np.random.uniform(0.1, 1.0),  # Ajuste basado en la fase del juego.
    'size': 9,  # Tamaño del tablero.
}

# Definir el estado inicial del tablero para un tablero de 9x9.
board = np.random.choice([-1, 0, 1], size=(params['size'], params['size']))

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*params['size']+j for i in range(params['size']) for j in range(params['size'])}

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: board[i, j] for i in range(params['size']) for j in range(params['size'])}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(params['size']):
    for j in range(params['size']):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < params['size']-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -params['J']
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -params['J']
        if j < params['size']-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -params['J']
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -params['J']

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=82b8d85e-ff20-40ba-aca5-c4e06d2b3d87

'http://127.0.0.1:18000/?problemId=82b8d85e-ff20-40ba-aca5-c4e06d2b3d87'

ISING con parámetros correlacionados
Este código está diseñado para resolver un problema de optimización cuántica utilizando la biblioteca D-Wave. Aquí está lo que hace cada parte del código:

Importa las bibliotecas necesarias.

Configura el tipo de solver y el token de D-Wave.

Inicializa el solver con la configuración especificada.

Define los parámetros base del modelo, que representan diferentes aspectos del problema de optimización.

Define el tamaño del tablero de juego.

Calcula los parámetros del modelo, algunos de los cuales están correlacionados con otros. Por ejemplo, M está directamente correlacionado con J, y R está directamente correlacionado con alpha.

Define el estado inicial del tablero de juego, que es un tablero de 9x9 con valores aleatorios de -1, 0, o 1.

Crea variables de spin para cada posición en el tablero.

Configura los valores de h de acuerdo con el estado inicial del tablero.

Construye el Hamiltoniano Ising, que es la función de energía para el problema de optimización.

Crea el modelo binario cuadrático, que es la representación del problema de optimización que se resolverá.

Crea un sampler, que es el objeto que resolverá el problema de optimización.

Muestrea el modelo, es decir, resuelve el problema de optimización.

Muestra el inspector, que es una herramienta de D-Wave para visualizar los resultados de la optimización.

Las variables correlacionadas en este código son M y R. M está directamente correlacionado con J, y R está directamente correlacionado con alpha. Esto significa que los valores de M y R dependen de los valores de J y alpha, respectivamente.

In [64]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros base del modelo
base_J = 0.5  # Interacción base entre piedras adyacentes.
base_alpha = 0.8  # Coeficiente base de decaimiento para la interacción a larga distancia.
base_L = 0.3  # Importancia base de las situaciones de Ko y Seki.
base_M = 0.7  # Importancia base de las condiciones de borde.
base_R = 0.6  # Ajuste base basado en la fase del juego.

# Tamaño del tablero
size = 9

# Correlación entre parámetros
J = base_J * np.random.uniform(0.9, 1.1)
alpha = base_alpha * np.random.uniform(0.9, 1.1)
L = base_L * np.random.uniform(0.9, 1.1)
M = base_M * J  # M está directamente correlacionado con J.
R = base_R * alpha  # R está directamente correlacionado con alpha.

# Definir el estado inicial del tablero para un tablero de 9x9.
board = np.random.choice([-1, 0, 1], size=(size, size))

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*size+j for i in range(size) for j in range(size)}

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: board[i, j] for i in range(size) for j in range(size)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(size):
    for j in range(size):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < size-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -J
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -J
        if j < size-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -J
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -J

# Crear el modelo binario cuadrático
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=f5fb5c38-0180-4631-a468-2f6104b3ef35

'http://127.0.0.1:18000/?problemId=f5fb5c38-0180-4631-a468-2f6104b3ef35'

In [65]:
# Usar D-Wave para encontrar la configuración con la energía más baja
response = sampler.sample(model)

# Imprimir la respuesta
print("Posición: 0")  # Solo hay un modelo, por lo que su posición es 0
for sample, energy in response.data(['sample', 'energy']):
    print(f"  Configuración: {sample}, Energía: {energy}")

Posición: 0
  Configuración: {0: -1, 1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 1, 7: 1, 8: 1, 9: -1, 10: -1, 11: -1, 12: -1, 13: -1, 14: -1, 15: 1, 16: 1, 17: 1, 18: -1, 19: -1, 20: -1, 21: -1, 22: -1, 23: -1, 24: 1, 25: 1, 26: 1, 27: -1, 28: -1, 29: -1, 30: -1, 31: -1, 32: -1, 33: 1, 34: 1, 35: 1, 36: -1, 37: -1, 38: -1, 39: -1, 40: -1, 41: -1, 42: 1, 43: 1, 44: 1, 45: -1, 46: -1, 47: -1, 48: -1, 49: -1, 50: -1, 51: 1, 52: 1, 53: 1, 54: -1, 55: -1, 56: -1, 57: -1, 58: -1, 59: -1, 60: 1, 61: 1, 62: 1, 63: -1, 64: -1, 65: -1, 66: -1, 67: -1, 68: -1, 69: 1, 70: 1, 71: 1, 72: -1, 73: -1, 74: -1, 75: -1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1}, Energía: -135.92005909347878
